In [ ]:
using Plots, Random

# Целевая функция
f(x) = 5 - 24x + 17x^2 - (11/3)x^3 + (1/4)x^4

# Новая структура особи с кэшированием f(x)
struct Individual
    x::Float64
    age::Int
    max_age::Int
    fitness::Float64
end

# Инициализация популяции
function initialize_population(size, x_min, x_max, min_age, max_age)
    pop = [create_individual(rand() * (x_max - x_min) + x_min, 0, min_age, max_age, []) for _ in 1:size]
    return pop
end

# Создание особи с вычисленным fitness и max_age
function create_individual(x, age, min_age, max_age, f_values)
    fit = f(x)
    ma = compute_max_age(fit, f_values, min_age, max_age)
    return Individual(x, age, ma, fit)
end

# Вычисление максимального возраста по нормализованному fitness
function compute_max_age(fit, f_values, min_age, max_age)
    if isempty(f_values)
        return min_age + (max_age - min_age) ÷ 2
    end
    f_min, f_max = extrema(f_values)
    normalized = f_max != f_min ? (f_max - fit) / (f_max - f_min) : 0.5
    return min_age + round(Int, normalized * (max_age - min_age))
end

x_min = 0.0
x_max = 5.0
p = 0.5
mutation_strength = 0.1
min_age = 1
max_age_param = 5
initial_pop_size = 50
generations = 50
max_pop_size = 100

# Инициализация
Random.seed!(42)
pop = initialize_population(initial_pop_size, x_min, x_max, min_age, max_age_param)

# Анимация
anim = @animate for gen in 1:generations
    global pop

    # Старение и фильтрация
    pop = [Individual(ind.x, ind.age + 1, ind.max_age, ind.fitness) for ind in pop]
    alive = [ind for ind in pop if ind.age <= ind.max_age]
    dead = [ind for ind in pop if ind.age > ind.max_age]
    pop_size_current = length(alive)

    # Потомки
    aux_pop_size = round(Int, pop_size_current * p)
    f_values = [ind.fitness for ind in alive]
    aux_pop = []

    if !isempty(alive)
        for _ in 1:aux_pop_size
            parents = rand(alive, 2)
            α = rand()
            x_child = α * parents[1].x + (1 - α) * parents[2].x
            x_child += randn() * mutation_strength
            x_child = clamp(x_child, x_min, x_max)
            push!(aux_pop, create_individual(x_child, 0, min_age, max_age_param, f_values))
        end
    end

    # Объединение и ограничение размера популяции
    pop = vcat(alive, aux_pop)
    if length(pop) > max_pop_size
        pop = sort(pop, by = x -> x.fitness)
        pop = pop[1:max_pop_size]
    end

    # Визуализация
    plot_x = range(x_min, x_max, length=200)
    plot_f = f.(plot_x)
    plt = plot(plot_x, plot_f, label="f(x)", xlim=(x_min, x_max), ylim=(minimum(plot_f)-5, maximum(plot_f)+5), legend=:topleft)

    if !isempty(pop)
        scatter!(plt, [ind.x for ind in pop], [ind.fitness for ind in pop], color=:blue, label="Current Pop")
    end
    if !isempty(aux_pop)
        scatter!(plt, [ind.x for ind in aux_pop], [ind.fitness for ind in aux_pop], color=:red, label="New Individuals")
    end
    if !isempty(dead)
        scatter!(plt, [ind.x for ind in dead], [ind.fitness for ind in dead], color=:black, marker=:x, label="Dead", markersize=8)
    end

    title!("Generation $gen | PopSize: $(length(pop))")
end

gif(anim, "genetic_algorithm_optimized.gif", fps=2)


┌ Info: Saved animation to c:\Users\Golum\Desktop\iu9-education\optimization-methods\letuchka-14-04-2025\genetic_algorithm_optimized.gif
└ @ Plots C:\Users\Golum\.julia\packages\Plots\kLeqV\src\animation.jl:156


MethodError: MethodError: no method matching findmin(::Vector{Any}; by::var"#109#110")

Closest candidates are:
  findmin(::AbstractArray; dims) got unsupported keyword argument "by"
   @ Base reducedim.jl:1130
  findmin(::Any, !Matched::SparseArrays.AbstractSparseVector{T}) where T got unsupported keyword argument "by"
   @ SparseArrays C:\Users\Golum\.julia\juliaup\julia-1.10.5+0.x64.w64.mingw32\share\julia\stdlib\v1.10\SparseArrays\src\sparsevector.jl:1610
  findmin(::Any, !Matched::AbstractArray; dims) got unsupported keyword argument "by"
   @ Base reducedim.jl:1153
  ...
